In [8]:
from transformers import DetrImageProcessor, DetrForObjectDetection
from PIL import Image
import torch
import os

# Initialize the model and processor
processor = DetrImageProcessor.from_pretrained("facebook/detr-resnet-101", revision="no_timm")
model = DetrForObjectDetection.from_pretrained("facebook/detr-resnet-101", revision="no_timm")

image_folder = r"E:\VS_Code\ZHAW\ML2\Image_class_test\relevant_images"
output_file_path = r"E:\VS_Code\ZHAW\ML2\Image_class_test\results_detection.txt"

# Specify the objects you want to detect
desired_objects = ["sign", "licenseplate", "car", "streetsign"]

# Open the output file in write mode
with open(output_file_path, 'w') as file:
    # Loop through each image in the folder
    for image_file in os.listdir(image_folder):
        image_path = os.path.join(image_folder, image_file)
        
        if image_path.lower().endswith(('.png', '.jpg', '.jpeg')):  # Check if it's an image file
            image = Image.open(image_path)
            inputs = processor(images=image, return_tensors="pt")
            outputs = model(**inputs)

            # Process detections
            target_sizes = torch.tensor([image.size[::-1]])
            results = processor.post_process_object_detection(outputs, target_sizes=target_sizes, threshold=0.9)[0]

            detections = []
            for score, label, box in zip(results["scores"], results["labels"], results["boxes"]):
                #if model.config.id2label[label.item()] in desired_objects:
                    box = [round(i, 2) for i in box.tolist()]
                    detection = (
                        f"Detected {model.config.id2label[label.item()]} with confidence "
                        f"{round(score.item(), 3)} at location {box}"
                    )
                    detections.append(detection)

            # Write the results for this image to the file, if any detections are found
            if detections:
                file.write(f"{image_file} :\n")
                for detection in detections:
                    file.write(f"  {detection}\n")
                file.write("\n")  # Add a new line for readability

print("Object detection completed and results saved.")


Object detection completed and results saved.
